In [10]:
import os
from pathlib import Path
project_root = os.path.join(str(Path.home()), 'research/qd/diffusion_models')
os.chdir(project_root)
%pwd # should be PPGA root dir

'/home/shashank/research/qd/diffusion_models'

In [11]:
from autoencoders.policy_hyperautoencoder import HyperAutoEncoder
from envs.brax_custom.brax_env import make_vec_env_brax

from dataset.policy_dataset import e_data_loader_test, env_cfg, actor_cfg

In [12]:
from dataset.policy_dataset import actor_cfg, Actor, test_elites, scheduler
import torch
from IPython.display import HTML, Image
from brax.io import html, image


In [13]:
env = make_vec_env_brax(env_cfg)

In [14]:
model = HyperAutoEncoder(actor_cfg, emb_channels=8, z_channels=4)
model_cp = './checkpoints/autoencoder.pt'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.load_state_dict(torch.load(model_cp))
model.to(device)
model.eval()

HyperAutoEncoder(
  (encoder): ModelEncoder(
    (dummy_actor): Actor(
      (obs_normalizer): NormalizeObservation(
        (obs_rms): RunningMeanStd()
      )
      (reward_normalizer): NormalizeReward(
        (return_rms): RunningMeanStd()
      )
      (actor_mean): Sequential(
        (0): Linear(in_features=18, out_features=128, bias=True)
        (1): Tanh()
        (2): Linear(in_features=128, out_features=128, bias=True)
        (3): Tanh()
        (4): Linear(in_features=128, out_features=6, bias=True)
      )
    )
    (out): Linear(in_features=4608, out_features=128, bias=True)
  )
  (decoder): MLP_GHN(
    (ln): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
    (embed): Embedding(3, 16)
    (shape_enc3): Linear(in_features=1, out_features=16, bias=True)
    (measure_enc): Linear(in_features=64, out_features=7, bias=True)
    (gnn): GatedGNN(
      (mlp): MLP(
        (fc): Sequential(
          (0): Linear(in_features=16, out_features=16, bias=True)
          (1): 

In [121]:
def enjoy_brax(agent, render=True, deterministic=True):
    if actor_cfg.normalize_obs:
        obs_mean, obs_var = agent.obs_normalizer.obs_rms.mean, agent.obs_normalizer.obs_rms.var

    obs = env.reset()
    rollout = [env.unwrapped._state]
    total_reward = 0
    measures = torch.zeros(env_cfg.num_dims).to(device)
    done = False
    while not done:
        with torch.no_grad():
            obs = obs.unsqueeze(dim=0).to(device)
            if actor_cfg.normalize_obs:
                obs = (obs - obs_mean) / torch.sqrt(obs_var + 1e-8)

            if deterministic:
                act = agent.actor_mean(obs)
            else:
                act, _, _ = agent.get_action(obs)
            act = act.squeeze()
            obs, rew, done, info = env.step(act.cpu())
            measures += info['measures']
            rollout.append(env.unwrapped._state)
            total_reward += rew
    if render:
        i = HTML(html.render(env.unwrapped._env.sys, [s.qp for s in rollout]))
        display(i)
    print(f'{total_reward=}')
    # print(f' Rollout length: {len(rollout)}')
    measures /= len(rollout)
    print(f'Recorded Measures: {measures.cpu().numpy()}')
    return total_reward.detach().cpu().numpy()

In [116]:
agent = Actor(actor_cfg, obs_shape=actor_cfg.obs_shape[0], action_shape=actor_cfg.action_shape, deterministic=True).to(device)

In [67]:
def get_random_elite():
    elite = scheduler.archive.sample_elites(1)
    agent = Actor(actor_cfg, obs_shape=actor_cfg.obs_shape[0], action_shape=actor_cfg.action_shape).deserialize(elite.solution_batch.flatten()).to(device)
    if actor_cfg.normalize_obs:
        agent.obs_normalizer = elite.metadata_batch[0]['obs_normalizer']
    return agent, elite.measures_batch[0], elite

In [100]:
agent, measure, elite = get_random_elite()
print(f'Sampled measure from elite: {measure}')


Sampled measure from elite: [0.98016679 0.94916666]


In [103]:
enjoy_brax(agent, render=False)

Recorded Measures: [0.97002995 0.97402596]


array(1105.4646, dtype=float32)

In [107]:
# weights_dict = agent.get_deserialized_weights(elite.solution_batch.flatten())
x,y = e_data_loader_test.__iter__().__next__()


In [108]:
reconstructed_models = model(x)

In [122]:
for i, recon_model in enumerate(reconstructed_models[0]):
    desired_measure = y[i]
    print(f'Desired measure: {desired_measure}')
    agent.actor_mean = recon_model
    enjoy_brax(agent, render=False)
    print('--------------------------------------------------------------------')

Desired measure: tensor([0.6312, 0.2800], device='cuda:0')
total_reward=tensor(-433.5556, device='cuda:0')
Recorded Measures: [0.24975024 0.18681318]
--------------------------------------------------------------------
Desired measure: tensor([0.0830, 0.7422], device='cuda:0')
total_reward=tensor(278.1645, device='cuda:0')
Recorded Measures: [0.03796204 0.16983016]
--------------------------------------------------------------------
Desired measure: tensor([0.6202, 0.2725], device='cuda:0')
total_reward=tensor(-58.8245, device='cuda:0')
Recorded Measures: [0.02797203 0.1958042 ]
--------------------------------------------------------------------
Desired measure: tensor([0.5358, 0.0772], device='cuda:0')
total_reward=tensor(350.6901, device='cuda:0')
Recorded Measures: [0.1038961  0.16483516]
--------------------------------------------------------------------
Desired measure: tensor([0.7707, 0.5513], device='cuda:0')
total_reward=tensor(147.8383, device='cuda:0')
Recorded Measures: [0